# Create animations of hormones over time

In [1]:
import setcwd

setcwd.main()

Working directory:  /Volumes/GoogleDrive/My Drive/code/my28brains/my28brains
Directory added to path:  /Volumes/GoogleDrive/My Drive/code/my28brains
Directory added to path:  /Volumes/GoogleDrive/My Drive/code/my28brains/my28brains


In [2]:
import os
import re

import pandas as pd

from joblib import Parallel, delayed

from my28brains.viz import plotly_hormones
from my28brains.viz import HORMONES, FIGS, ANIMS, TMP

In [3]:
print("HORMONES = ", HORMONES)
print("FIGS = ", FIGS)
print("ANIMS = ", ANIMS)
print("TMP = ", TMP)

HORMONES =  {'Estro': 'Estrogen', 'Prog': 'Progesterone', 'LH': 'LH', 'FSH': 'FSH'}
FIGS =  /Volumes/GoogleDrive/My Drive/code/my28brains/results/figs
ANIMS =  /Volumes/GoogleDrive/My Drive/code/my28brains/results/anims
TMP =  /Volumes/GoogleDrive/My Drive/code/my28brains/results/tmp


## Load the hormones data

In [4]:
DATA_PATH = os.path.join(os.path.dirname(os.getcwd()), "data", "hormones.csv")
df = pd.read_csv(DATA_PATH, delimiter=",")
print(f"df has length: {len(df)} days")
df.head()

df has length: 60 days


,dayID,Estro,Prog,Test,DHEAS,LH,FSH,SHBG,EthinylEstradiol,Levonorgestresl,CycleDay
0,1,52.4,11.9,98.7,192.88,8.61,5.06,55.72,NaN,NaN,21
1,2,74.2,10.6,81.2,247.12,4.89,4.13,58.22,NaN,NaN,22
2,3,91.7,12.4,96.3,226.23,5.98,3.49,56.92,NaN,NaN,23
3,4,94.6,13.5,90.5,212.84,4.32,3.65,57.29,NaN,NaN,24
4,5,110.0,15.0,77.6,217.60,3.44,3.30,58.87,NaN,NaN,25


## Select the cycle of interest: natural or OC


We select the first 30 days to get the first (natural) cycle.

In [5]:
natural_cycle_df = df[df["dayID"] < 31]
n_days = len(natural_cycle_df)
print(f"natural_cycle_df has length: {len(natural_cycle_df)} days.")

natural_cycle_df has length: 30 days.


## Make the animation

Make one plot to check the design.

In [6]:
CONFIG = {
    "all": {"hormones": HORMONES, "ymax": natural_cycle_df["Estro"].max() + 10},
    "prog": {"hormones": ["Prog"], "ymax": natural_cycle_df["Prog"].max() + 5},
}

In [7]:
to_plot = "prog"
by = "CycleDay"
framerate = 2

plotly_hormones(
    natural_cycle_df,
    by=by,
    day=20,
    hormones=CONFIG[to_plot]["hormones"],
    ymax=CONFIG[to_plot]["ymax"],
    savefig=False,
)

Make all plots to create the many .pngs.

In [8]:
def plot_all(i):
    plotly_hormones(
    natural_cycle_df,
    by=by,
    day=i,
    hormones=CONFIG[to_plot]["hormones"],
    ymax=CONFIG[to_plot]["ymax"],
    savefig=True,
    )


_ = Parallel(n_jobs=-1)(delayed(plot_all)(i) for i in range(1, n_days))

Use ffmpeg to make the animation

In [9]:
by_lower = re.sub(r'(?<!^)(?=[A-Z])', '_', by).lower()
input_pattern = os.path.join(TMP, f"hormones_day_*.png")
palette_path = os.path.join(TMP, "palette.png")

output_name = f"{to_plot}_by_{by_lower}_framerate{framerate}"

output_gif = os.path.join(ANIMS, f"{output_name}.gif")
output_mov = os.path.join(ANIMS, f"{output_name}.mov")

In [10]:
cmd = f"ffmpeg -y -framerate {framerate} -pattern_type glob -i '{input_pattern}' -vf palettegen '{palette_path}'"
os.system(cmd)

ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox

0

In [11]:
# Create GIF
cmd_gif = (
    f"ffmpeg -y -framerate {framerate} "
    f"-pattern_type glob -i '{input_pattern}' -i '{palette_path}' -filter_complex \"paletteuse\" '{output_gif}'"
)
os.system(cmd_gif)

# Create MOV
cmd_mov = (
    f"ffmpeg -y -framerate {framerate} "
    f"-pattern_type glob -i '{input_pattern}' -c:v libx264 -pix_fmt yuv420p -crf 23 '{output_mov}'"
)
os.system(cmd_mov)

ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox

0